# Prueba.

In [5]:
from typing import Dict, Tuple

import pandas as pd
from sklearn.datasets import load_iris

from genesis_explorer.makers.split_data_maker import SplitDataMaker


@SplitDataMaker.split_data_task(experiment_name = "split-data-experiment",
                                experiment_version = "1.0.0-test.4")
def split_data_task(data: pd.DataFrame) -> Dict[str, Tuple[pd.DataFrame, ...]]:
    """Función que se encarga de separar los datos en train y test."""

    return {"train": data.iloc[:-100], "test": data.iloc[-100:]}


data = load_iris()
df = pd.DataFrame(data["data"], columns = data["feature_names"])

split_data_task(data = df)

TypeError: SplitDataMaker.split_data_task() missing 3 required positional arguments: 'cloud_provider', 'project_id', and 'cloud_uri'

In [2]:
from genesis_explorer.artifacts.context import ArtifactContextManager


cloud_provider = "gcp"
cloud_uri = "gs://migracion-gcp-bucket/consumers/consumers-iris/mlb/1/"
project_id = "mlops-credits-vertex-poc"


with ArtifactContextManager(cloud_provider = cloud_provider,
                            cloud_uri = cloud_uri,
                            project_id = project_id) as manager:
                            
    artifact = manager.download(file_name = "test.pickle")

2025-11-15 10:21:39,129 - INFO - Esta clase implementa el patrón Singleton, por ende, se retorna la instancia existente.
2025-11-15 10:21:39,130 - INFO - Descargando el archivo consumers/consumers-iris/mlb/1/test.pickle del bucket migracion-gcp-bucket en el proyecto mlops-credits-vertex-poc...
2025-11-15 10:21:39,398 - INFO - Archivo consumers/consumers-iris/mlb/1/test.pickle descargado correctamente.


In [3]:
"""Módulo que contiene la lógica de la task de separación de datos."""

# Librerías Externas.
from typing import List, Dict, Tuple, Optional

import pandas as pd
from sklearn.model_selection import train_test_split

from genesis_explorer.makers.split_data_maker import SplitDataMaker


class MyDataSplitter(SplitDataMaker):
    """Clase que contiene la lógica para, de una tabla base, generar una separación de datos
    para el modelado."""

    def __init__(self,
                 data: List[str],
                 test_size: Optional[float] = 0.3,
                 oot_size: Optional[float] = 0.2,
                 random_seed: Optional[int] = 42,
                 cloud_provider: Optional[str] = "gcp",
                 project_id: Optional[str] = "mlops-credits-vertex-poc",
                 cloud_uri: Optional[str] = "gs://migracion-gcp-bucket/consumers/consumers-iris/mlb/1/",
                 experiment_name: Optional[str] = "split-data-experiment",
                 experiment_version: Optional[str] = "1.0.0-test.200") -> None:
        """Método de instanciación de la clase.
        
        Args:
        ----------
        test_size: Optional[float].
            Tamaño de la partición de test.

        oot_size: Optional[float].
            Tamaño de la partición de oot.

        random_seed: Optional[int].
            Semilla para la generación de números aleatorios."""
        
        super().__init__(cloud_provider = cloud_provider,
                         cloud_uri = cloud_uri,
                         project_id = project_id)
        
        artifacts = self.load_artifacts(artifact_names = data)
        self.data = artifacts["train_set"]

        self.oot_size = oot_size
        self.test_size = test_size
        self.random_seed = random_seed

        self.experiment_name = experiment_name
        self.experiment_version = experiment_version

    def run_task(self) -> None:
        """Método de ejecución de la task."""

        decorated_splitting_process = self.split_data_task(experiment_name = self.experiment_name,
                                                           experiment_version = self.experiment_version)(self.split_data)
        
        decorated_splitting_process(data = self.data)
    
    def split_data(self, data: pd.DataFrame) -> Dict[str, Tuple[pd.DataFrame, ...]]:
        """Función que se encarga de separar los datos en train y test."""

        dataframe_train, dataframe_test = train_test_split(data,
                                                           test_size = self.test_size,
                                                           random_state = self.random_seed)
        
        dataframe_test, dataframe_oot = train_test_split(dataframe_test,
                                                         test_size = self.oot_size,
                                                         random_state = self.random_seed)
        
        datasets = {"train_set": dataframe_train,
                    "test_set": dataframe_test,
                    "oot_set": dataframe_oot}

        return datasets
    

import pandas as pd
from sklearn.datasets import load_iris

data = load_iris()
df = pd.DataFrame(data["data"], columns = data["feature_names"])

my_data_splitter = MyDataSplitter(data = ["train_set"],
                                  experiment_version = "1.0.0-test.202")
my_data_splitter.run_task()

2025-11-15 10:50:28,420 - INFO - Esta clase implementa el patrón Singleton, por ende, se retorna la instancia existente.
2025-11-15 10:50:28,420 - INFO - Descargando el archivo consumers/consumers-iris/mlb/1/train_set.pickle del bucket migracion-gcp-bucket en el proyecto mlops-credits-vertex-poc...
2025-11-15 10:50:28,672 - INFO - Archivo consumers/consumers-iris/mlb/1/train_set.pickle descargado correctamente.
2025-11-15 10:50:28,676 - INFO - Esta clase implementa el patrón Singleton, por ende, se retorna la instancia existente.
2025-11-15 10:50:28,677 - INFO - Creando experimento: split-data-experiment
2025-11-15 10:50:29,905 - INFO - Experimento ya existe: split-data-experiment
2025-11-15 10:50:30,477 - INFO - Creando ExperimentRun: experimento-v202
2025-11-15 10:50:32,286 - INFO - ExperimentRun creado: experimento-v202
2025-11-15 10:50:32,287 - INFO - Iniciando el proceso de partición de datos...

2025-11-15 10:50:32,288 - INFO - Particionando los datos...
2025-11-15 10:50:32,292 -